In [1]:
from flask import Flask, render_template, request
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

app = Flask(__name__)

dic = {0 : 'No Tumor Found ;)', 1 : 'Found Pituitary Tumor',2:'Found Meningioma Tumor',3:'Found Glioma Tumor'
,4:'No Tumor Found ;)',5:'Found Pituitary Tumor',6:'Found Meningioma Tumor',7:'Found Glioma Tumor'
       ,8:'No Tumor Found ;)',9:'Found Pituitary Tumor',10:'Found Meningioma Tumor',11:'No Tumor Found ;)'}

model = load_model('kaggle_model_2.h5')

model.make_predict_function()

def predict_label(img_path):
    print("Aliiiiiii")
    i=image.image_utils.load_img(img_path, target_size=(150, 150))
    #i = image.load_img(img_path, target_size=(100,100))
    #i = image.img_to_array(i)/255.0
    i=image.image_utils.img_to_array(i)/255.0
    i = i.reshape(-1,150, 150, 1)
    p = model.predict(i)
    p2  = np.argmax(p,axis=1)
    
    print(p)
    print(np.argmax(p))
    
    return dic[np.argmax(p)]
    '''x=[p[0][0],p[1][0],p[2][0],p[0][1],p[1][1],p[2][1],p[0][2],p[1][2],p[2][2],p[0][3],p[1][3],p[2][3]]
    if max(x)== p[0][0] or max(x)==p[1][0]or max(x)==p[2][0]:
        s= 0
        return dic[s]
    elif max(x)==p[0][1] or max(x)==p[1][1] or max(x)==p[2][1]:
        s= 1
        return dic[s]

    elif max(x)==p[0][2] or max(x)==p[1][2] or max(x)==p[2][2]:
        s= 2
        return dic[s]
 
    elif max(x)==p[0][3] or max(x)==p[1][3] or max(x)==p[2][3]:
        s= 3            
        return dic[s]'''


# routes
@app.route("/", methods=['GET', 'POST'])
def main():
    return render_template("predcit.html")

@app.route("/main", methods=['GET', 'POST'])
def test():
    return render_template("base.html")


@app.route("/submit", methods = ['GET', 'POST'])
def get_output():
    if request.method == 'POST':
        #print(request.files)
        img = request.files['file']

        img_path = "static/" + img.filename
        print(request.files)
        img.save(img_path)

        p = predict_label(img_path)
        print("AAAAAAAAAAA")
        return render_template("predcit.html", prediction = p, img_path = img_path)
    return render_template("predcit.html")


if __name__ =='__main__':
    #app.debug = True
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Nov/2022 19:53:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2022 20:00:10] "POST /ProcessUserinfo/$%7BJSON.stringify(userImage)%7D HTTP/1.1" 404 -


ImmutableMultiDict([('file', <FileStorage: 'image (4).jpg' ('image/jpeg')>)])
Aliiiiiii
1/1 [==============================] - 0s 318ms/step


127.0.0.1 - - [17/Nov/2022 20:00:21] "POST /submit HTTP/1.1" 200 -


[[3.1256748e-03 1.6010251e-02 6.2949926e-01 3.5136479e-01]
 [2.4515089e-06 1.4706563e-06 1.4110637e-01 8.5888976e-01]
 [9.7801039e-06 1.1656309e-08 5.8456226e-06 9.9998438e-01]]
11
AAAAAAAAAAA


127.0.0.1 - - [17/Nov/2022 20:00:21] "GET /static/image%20(4).jpg HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2022 20:00:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2022 20:00:34] "POST /ProcessUserinfo/$%7BJSON.stringify(userImage)%7D HTTP/1.1" 404 -


ImmutableMultiDict([('file', <FileStorage: 'image (11).jpg' ('image/jpeg')>)])
Aliiiiiii
1/1 [==============================] - 0s 104ms/step


127.0.0.1 - - [17/Nov/2022 20:00:45] "POST /submit HTTP/1.1" 200 -


[[1.1404902e-04 1.3691273e-06 9.9948800e-01 3.9657406e-04]
 [6.4218722e-07 1.7147204e-04 2.8166693e-01 7.1816093e-01]
 [1.0799354e-06 4.2119439e-04 2.2946822e-05 9.9955469e-01]]
11
AAAAAAAAAAA


127.0.0.1 - - [17/Nov/2022 20:00:45] "GET /static/image%20(11).jpg HTTP/1.1" 200 -


In [2]:
150*150

22500

In [1]:
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
from tensorflow.keras.preprocessing import image
app = Flask(__name__)
model = load_model('kaggle_model_2.h5')
target_img = os.path.join(os.getcwd() , 'static/img')
@app.route('/')
def index_view():
    return render_template('index2.html')
#Allow files with extension png, jpg and jpeg
ALLOWED_EXT = set(['jpg' , 'jpeg' , 'png'])
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXT
           
# Function to load and prepare the image in right shape
def read_image(filename):
   img = load_img(filename, target_size=(224, 224))
   x = image.img_to_array(img)
   x = np.expand_dims(x, axis=0)
   x = preprocess_input(x)
   return x


@app.route('/predict',methods=['GET','POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename): #Checking file format
            filename = file.filename
            file_path = os.path.join('static/images', filename)
            file.save(file_path)
            img = read_image(file_path) #prepressing method
            class_prediction=model.predict(img) 
            classes_x=np.argmax(class_prediction,axis=1)
            if classes_x == 0:
              fruit = "no"
            elif classes_x == 1:
              fruit = "1"
            elif classes_x == 2:
                  fruit = "2"
            else:
              fruit = "3"
            #'fruit' , 'prob' . 'user_image' these names we have seen in predict.html.
            return render_template('predict.html', fruit = fruit,prob=class_prediction, user_image = file_path)
        else:
            return "Unable to read the file. Please check file extension"
if __name__ == '__main__':
    app.run(debug=True,use_reloader=False, port=8000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [16/Nov/2022 18:40:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2022 18:41:03] "POST /submit HTTP/1.1" 404 -
